# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs vroom iinput output formats:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

http://localhost/notebooks/tests/test5.ipynb 2 veicoli 1 solo mattina 2 corse

http://localhost/notebooks/tests/test6.ipynb 2 veicoli tutto il giorno 4 corse con priorita


In [13]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time
import polyline

vroom_url = 'http://vroom:3000/optimization/'
vehicleProfile = "driving-car"

zoom = 14

point_start =  [11.10249,46.21579] #posizione iniziale veicoli
C = [
      [7.495079040527344,44.34349388385857],
      [7.584342956542982,44.34300281878521],
      [7.585029602050781,44.42617960691271],
      [7.495079040527344,44.42495366933076],
      [7.495079040527344,44.34349388385857]
    ]
##Cuneo polygon coordinates

Cstart =  [7.574901580810547,44.407910474534425]
mapBbox = [C[0][0],C[0][1],C[1][0],C[2][1]]

point_start = Cstart.copy()

shipsProgrammed = [1,2]    # SIMULA ELENCO PRENOTAZIONI PROGRAMMATE( elenco id prenotazioni)
GENERATED_SHIPS = 5        #NUMERO SHIPMENTS GENERATE

center = point_start.copy()
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

def randomPoint(bb):
    min_x, min_y, max_x, max_y = bb
    x = random.uniform(min_x, max_x)
    y = random.uniform(min_y, max_y)
    return [x, y]

def randomColor():
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())

def vehicle2Map(v):
    idv = str(v['id'])
    caps = map(str, v['capacity'])

    try:
        loc = v['start']
        times = v['time_window']
    except:
        loc = None
        times = []
    
    mText = "VEICOLO %s" % idv
    mText += "<br> capacity %s" % ", ".join(caps)
    
    if len(times) > 0:
        mText += "<br>Orario turno <br> da %s a %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = COLORS[idv]
    
    if loc is not None:
        mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
        mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = j['location']
    if hasattr(j,'time_windows'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "da: %s <br> a: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right', permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):

    color = COLORS[str(s['pickup']['id'])]
    
    # prenotazione PARTENZA
    loc1 = s['pickup']['location']
    mText = "ID %s" % s['pickup']['id']
    if s['priority']>0:
        mText +=" (CONFERMATA)"
    
    try:
        times = s['pickup']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "<br>Partenza orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    #mText += "Quantita %s <br>" % s['amount'][0]
    #mText += "Priority %s <br>" % s['priority']
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True) 
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc1), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)
    
    # prenotazione ARRIVO
    loc2 = s['delivery']['location']
    mText = "ID %s <br>" % s['delivery']['id']
    try:
        times = s['delivery']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "Arrivo orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )

    mText += "Quantita %s <br>" % ", ".join(map(str, s['amount']))
    
    folium.PolyLine(locations = [revcc(loc1),revcc(loc2)], color = color, opacity = 0.6, weight = 3).add_to(Map)
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True)
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc2), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)

def step2Map(step):
    sid = ''
    wait = 0
    try:
        loc = step['location']
        sid = step['id']
        wait = step['waiting_time']
    except:
        loc = None

    if loc and step['type'] != 'end':
        tit = str(i)+" "+step['type']+"<br> arrivo: %s " % sec2hour(step['arrival'])

        mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(tit, direction='bottom', permanent=True, style='background:yellow'), icon=folium.Icon(icon='circle', prefix='fa', color='green'))
        mark.add_to(Map)

def step2Print(step):
    sid = ''
    wait = 0
    try:
        loc = step['location']
        sid = step['id']
        wait = step['waiting_time']
    except:
        loc = None

    if sid:
        sText = "CORSA %s alle %s - %s" % ( sid, sec2hour(step['arrival']), step['type'] )
    else:
        sText = "%s alle %s " % (step['type'].upper(), sec2hour(step['arrival']) )

    if wait :
        sText += " - Attesa %s " % str(sec2hour(wait))
    
    print(sText)

def route2Map(route):
    geomdecoded = ors.convert.decode_polyline(route['geometry'])
    folium.PolyLine(
        locations = [revcc(coords) for coords in geomdecoded['coordinates']],
        color = rcolor,
        opacity = 0.8,
        weight = 5
    ).add_to(Map)

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

COLORS = {
'1':'gray','2':'darkblue','3':'black','4':'beige','5':'lightgreen',
'6':'white','7':'darkpurple','8':'lightgray','9':'purple','10':'red',
'11':'green','12':'orange','13':'darkred','14':'pink','15':'cadetblue',
'16':'blue','17':'darkgreen','18':'lightblue','19':'lightred'
}

HOURS1 = [
[hour2sec('08:00'),hour2sec('08:05')],
[hour2sec('09:00'),hour2sec('09:05')],
[hour2sec('10:00'),hour2sec('10:05')],
[hour2sec('11:00'),hour2sec('11:05')],
[hour2sec('12:00'),hour2sec('12:05')]
]

HOURS2 = [
[hour2sec('14:00'),hour2sec('14:05')],
[hour2sec('15:00'),hour2sec('15:05')],
[hour2sec('16:00'),hour2sec('16:05')],
[hour2sec('17:00'),hour2sec('17:05')],
[hour2sec('18:00'),hour2sec('18:05')]
]
# VEICOLI

HOURSall = HOURS1 + HOURS2

vehicles = []

### VEICOLO 1
vid = 1

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [2,3],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': [              #orario lavoro del veicolo 1
        hour2sec('07:00'),
        hour2sec('13:00')
    ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2

loc = point_start

vehicle = {
    'id': vid,
    'capacity': [4,5],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    'time_window': [                  # pomeridiano
        hour2sec('04:00'),
        hour2sec('19:00')
    ]
}
vehicles.append(vehicle)

shipments = []

for i in list(range(GENERATED_SHIPS)):
    
    sid = i+1
    
    ship = {          #struttura minima valida per prenotazione
        'pickup': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'delivery': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'priority': 0,
        'amount': [ random.randint(4,6), random.randint(4,6) ]        
    }
    
    if sid in shipsProgrammed:
        ship['priority']= 100;

    shiptype = random.choice(['pickup','delivery']) # pickup == PARTENZA, delivery == ARRIVO
    
    ship[ shiptype ]['time_windows'] = [ random.choice(HOURSall) ]
    
    shipments.append(ship)

#vehicles = [vehicles[0]]    #solo primo veicolo

request = {
     'vehicles': vehicles,
     'shipments': shipments,
     'jobs': [],
}

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for v in request['vehicles']:
    vehicle2Map(v)

#print(vehicles)

#print(shipments)

for s in request['shipments']:
    
    ship2Map(s)
    timeStart = '     '
    timeEnd = '     '
    confirmed = ''
    amount = ''

    try:
        timeStart = sec2hour(s['pickup']['time_windows'][0][0])
    except:
        timeStart = '     '
    
    try:
        timeEnd = sec2hour(s['delivery']['time_windows'][0][0])
    except:
        timeEnd = '     '
    
    try:
        if s['priority'] > 0:
            confirmed='confermata'
    except:
        confirmed = ''
        
    amount = ",".join(map(str, s['amount']))
    
    shipText = "CORSA %s - quantita %s partenza %s arrivo %s %s" % (s['pickup']['id'], amount, timeStart, timeEnd, confirmed)
    print(shipText)

Map

CORSA 1 - quantita 4,4 partenza 16:00 arrivo       confermata
CORSA 2 - quantita 6,5 partenza       arrivo 10:00 confermata
CORSA 3 - quantita 5,6 partenza 09:00 arrivo       
CORSA 4 - quantita 6,5 partenza       arrivo 08:00 
CORSA 5 - quantita 5,5 partenza 11:00 arrivo       


In [14]:
request['geometry'] = True
solution = requests.post(vroom_url, json = request).json()

for v in request['vehicles']:
    vehicle2Map(v)

print(len(shipments), "CORSE richieste\n")
print("CORSE programmate:", shipsProgrammed )

if len(solution['routes']) > 0:
    for route in solution['routes']:
        #rcolor = COLORS[str(route['vehicle'])]
        rcolor = randomColor()
        rtext = "distance: %s delivered quantity: %s pickup quantity: %s "  % ( toKm(route['distance']), route['delivery'][0], route['pickup'][0] )
        rtext = rtext+", durata: %s " % toMin(route['duration']+route['service'])
        print("\nVehicle "+str(route['vehicle']), rcolor)
        print(rtext)
        route2Map(route)
        for step in route['steps']:
            step2Print(step)
            step2Map(step)

    print("\nCORSE NON ASSEGNATE:")
    for un in solution['unassigned']:
        print('CORSA '+str(un['id']))
    
    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))
    
Map

5 CORSE richieste

CORSE programmate: [1, 2]

Vehicle 2 #79B040
distance: 25km delivered quantity: 4 pickup quantity: 4 , durata: 33min 
START alle 15:48 
CORSA 1 alle 16:00 - pickup
CORSA 1 alle 16:12 - delivery
END alle 16:21 

CORSE NON ASSEGNATE:
CORSA 5
CORSA 5
CORSA 4
CORSA 4
CORSA 2
CORSA 2
CORSA 3
CORSA 3

Total distance: 25km Not Delivered: 8


In [6]:
#for route in solution['routes']:
#    route['geometry'] = ors.convert.decode_polyline(route['geometry'])
#    print(json.dumps(route['steps'], indent=2))

print(json.dumps(solution, indent=2))


{
  "code": 0,
  "summary": {
    "cost": 0,
    "unassigned": 10,
    "delivery": [
      0
    ],
    "amount": [
      0
    ],
    "pickup": [
      0
    ],
    "service": 0,
    "duration": 0,
    "waiting_time": 0,
    "priority": 0,
    "distance": 0,
    "computing_times": {
      "loading": 42,
      "solving": 0,
      "routing": 0
    }
  },
  "unassigned": [
    {
      "id": 5,
      "location": [
        7.511682454064316,
        44.34811846283789
      ]
    },
    {
      "id": 5,
      "location": [
        7.534805925345752,
        44.388120121518575
      ]
    },
    {
      "id": 4,
      "location": [
        7.539305860376324,
        44.352856957899355
      ]
    },
    {
      "id": 4,
      "location": [
        7.500576362705121,
        44.35906579445045
      ]
    },
    {
      "id": 1,
      "location": [
        7.532148932974997,
        44.40755983601501
      ]
    },
    {
      "id": 1,
      "location": [
        7.557949411594967,
        44.

In [6]:
print(json.dumps(request, indent=2))


{
  "vehicles": [
    {
      "id": 1,
      "capacity": [
        1
      ],
      "profile": "driving-car",
      "start": [
        7.578901580810546,
        44.41191047453442
      ],
      "end": [
        7.578901580810546,
        44.41191047453442
      ],
      "time_window": [
        1620802800,
        1620824400
      ]
    },
    {
      "id": 2,
      "capacity": [
        1
      ],
      "profile": "driving-car",
      "start": [
        7.574901580810547,
        44.407910474534425
      ],
      "end": [
        7.574901580810547,
        44.407910474534425
      ],
      "time_window": [
        1620792000,
        1620846000
      ]
    }
  ],
  "shipments": [
    {
      "pickup": {
        "id": 1,
        "location": [
          7.647060199651438,
          44.40783746854293
        ]
      },
      "delivery": {
        "id": 1,
        "location": [
          7.619097878068527,
          44.317713619379376
        ],
        "time_windows": [
          [
    